In [0]:
#primeiro baixar os filmes do movelens
#contem 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users.

In [0]:
#https://grouplens.org/datasets/movielens/latest/

import pandas as pd

#ler o arquivo
filmes = pd.read_csv('movies.csv')

#sobre escrever o cabesalho
filmes.columns = ['filmeId','titulo', 'generos']

#subistuir o indice pela coluna de id
filmes = filmes.set_index("filmeId")

#retorna os 5 primeiros itens
filmes.head()

,titulo,generos
filmeId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [0]:
filmes.describe()

,total_votos,nota_media
count,9724.000000,9724.000000
mean,10.369807,3.262448
std,22.401005,0.869874
min,1.000000,0.500000
25%,1.000000,2.800000
50%,3.000000,3.416667
75%,9.000000,3.911765
max,329.000000,5.000000


In [0]:
notas = pd.read_csv('ratings.csv')
notas.columns = ('usuarioId', 'filmesId', 'nota', 'data')
notas.head()

,usuarioId,filmesId,nota,data
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [0]:
#descrisao da tabela media, max, min, quantidade
notas.describe()

,usuarioId,filmesId,nota,data
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


##1 tentativa de recomendacao

####sabendo que nao sei nada sobre a pessoa, porem eu tenho informacoes sobre os filmes

####popularidade é os filmes mais votados mas nao com as melhores notas ou os filmes mais vistos

In [0]:
#counts conta a frequencia assim vamos recomenda o mais popular
#uma heuristica sem provar funcionam melhor != popular
total_votos = notas['filmesId'].value_counts()
total_votos.head()

356     329
318     317
296     307
593     279
2571    278
Name: filmesId, dtype: int64

In [0]:
filmes.loc[356]

titulo          Forrest Gump (1994)
generos    Comedy|Drama|Romance|War
Name: 356, dtype: object

In [0]:
#incluir colunas na tabela para facilitar, pandas usa o indice
filmes["total_votos"] = total_votos
filmes.head()

#ordena pelo valor, por padrao o panda eé ascending == true
filmes.sort_values("total_votos", ascending = False).head()

,titulo,generos,total_votos
filmeId,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0


##2 heuristicas

####ordenar pela nota media e filtra q tem maior q a media de quantidade de notas

####problemas de ter apenas uma coluna para recomendar eé filmes de nicho

In [0]:
#pegas as notas medias do filmes
notas_medias = notas.groupby('filmesId').mean()['nota']

notas_medias.head()

filmesId
1    3.920930
2    3.431818
3    3.259615
4    2.357143
5    3.071429
Name: nota, dtype: float64

In [0]:
#adicionar na tabela a nota media
filmes['nota_media'] = notas_medias
filmes.head()

,titulo,generos,total_votos,nota_media
filmeId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818
3,Grumpier Old Men (1995),Comedy|Romance,52.0,3.259615
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0,2.357143
5,Father of the Bride Part II (1995),Comedy,49.0,3.071429


In [0]:
#nem sempre o mais votado tem a melhor nota
filmes.sort_values("total_votos", ascending = False).head()

,titulo,generos,total_votos,nota_media
filmeId,,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0,4.161290
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446


In [0]:
#nem sempre o com a nota mais alta esta certo por conta da quantidade
filmes.sort_values("nota_media", ascending = False).head()

,titulo,generos,total_votos,nota_media
filmeId,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0


In [0]:
#filtra de quantidade maior que a media
filmes_50mais = filmes.query('total_votos >= 50')
filmes_50mais.sort_values("nota_media", ascending = False).head()

,titulo,generos,total_votos,nota_media
filmeId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041


In [0]:
'''REFERENCIAS

threshold User neighborhood in python
https://scikit-learn.org/stable/modules/neighbors.html
https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/
https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8
https://gurus.pyimagesearch.com/lesson-sample-k-nearest-neighbor-classification/
https://www.pyimagesearch.com/2015/12/07/local-binary-patterns-with-python-opencv/
https://commons.wikimedia.org/wiki/File:Python_3._The_standard_type_hierarchy.png
https://datascienceplus.com/predict-customer-churn-logistic-regression-decision-tree-and-random-forest/



https://grouplens.org/datasets/movielens/latest/
https://scikit-learn.org/stable/
https://spark.apache.org/docs/latest/ml-guide.html
http://muricoca.github.io/crab/
https://en.wikipedia.org/wiki/Singular_value_decomposition
https://code.google.com/archive/p/pysuggest/
http://glaros.dtc.umn.edu/gkhome/suggest/overview


https://colab.research.google.com/notebooks/welcome.ipynb#scrollTo=-gE-Ez1qtyIA
https://colab.research.google.com/notebooks/charts.ipynb
https://colab.research.google.com/notebooks/mlcc/tensorflow_programming_concepts.ipynb
https://colab.research.google.com/notebooks/tpu.ipynb

#https://stackoverflow.com/questions/4819437/javas-mahout-equivalent-in-python
'''
#verificar a versao python3 -v

'REFERENCIAS\nhttps://grouplens.org/datasets/movielens/latest/\nhttps://scikit-learn.org/stable/\nhttps://spark.apache.org/docs/latest/ml-guide.html\nhttp://muricoca.github.io/crab/\nhttps://en.wikipedia.org/wiki/Singular_value_decomposition\nhttps://code.google.com/archive/p/pysuggest/\nhttp://glaros.dtc.umn.edu/gkhome/suggest/overview\n\n\nhttps://colab.research.google.com/notebooks/welcome.ipynb#scrollTo=-gE-Ez1qtyIA\nhttps://colab.research.google.com/notebooks/charts.ipynb\nhttps://colab.research.google.com/notebooks/mlcc/tensorflow_programming_concepts.ipynb\nhttps://colab.research.google.com/notebooks/tpu.ipynb\n\n#https://stackoverflow.com/questions/4819437/javas-mahout-equivalent-in-python\n'